# YANG for Dummies!

Most people are excited to hear about the new projects from NetworkToCode, but quickly discover that there are a lot of moving parts that present a steep learning curve.  Community contributions will be critical to the success and adoption of these projects.  The groundwork has already been laid for IOS and JunOS drivers, so I want to document the path to onboarding completely new drivers.  I'll demonstrate this based on David Barroso's great article titled ["YANG for dummies"](https://napalm-automation.net/yang-for-dummies/).  

- Create yang model in `./yang/dummies/models/star-wars/napalm-star-wars.yang`
  - ```
    dummies
    └── models
       ├── napalm-star-wars
       │  └── napalm-star-wars.yang
       └── napalm-star-wars-library.json
    ```
    - Copied from [Napalm-Automation](https://napalm-automation.net/yang-for-dummies/)
  - Create the library file (minus the comment): `cat napalm-star-wars-library.json`
    ```
    {
        "ietf-yang-library:modules-state": {
            "module-set-id": "6bd894f2-9168-484e-a0bf-f3ed38d864f9",
            "module": [
                {
                    "name": "napalm-star-wars",
                    "revision": "2019-08-31",  # <- This key *must* be present, see RFC8040
                    "conformance-type": "implement"
                }
            ]
        }
    }
    ```
    [RFC8040](https://tools.ietf.org/html/rfc8040)
  - `head napalm-star-wars.yang`
    ```
    // module name
    module napalm-star-wars {

        // boilerplate
        yang-version "1";
        namespace "https://napalm-yang.readthedocs.io/yang/napalm-star-wars";

        prefix "napalm-star-wars";
        revision "2019-08-31" {  # <- This *must* match the revision above.
          description
              "initial version";
          reference "0.0.1";
        }
        ...
    ```
- Update module paths in `./ntc_rosetta/yang/__init__.py`
- Create parser class in `./ntc_rosetta/parsers/dummies/dummy/__init__.py`
  - Directory layout
    ```
    parsers/dummies <- YANG namespace (like ntc or openconfig)
    ├── __init__.py
    └── dummy  <- Device driver name (like ios or junos)
       ├── __init__.py
       └── napalm_star_wars
          ├── __init__.py
          └── star_wars.py  <- parser logic load YANG instance of data model
    ```
- Add Parser class to `./ntc_rosetta/parsers/__init__.py`
  - `from ntc_rosetta.parsers.dummies.dummy import DummyParser`
- TODO: actually write the parser
- TODO: translator foo
- Add driver to `./ntc_rosetta/drivers/[driver_name].py`
- TODO: tests

In [9]:
from ntc_rosetta import get_driver

dummy = get_driver("dummy", "napalm_star_wars")
dummy_driver = dummy()

## The dummy device
Our dummy device is going to implement configuration through YAML files.  That's absurd you say!  YAML is a horrible format!  I don't disagree, but it's suitable for pet examples and it lives up to the "dummy" device name.

In [10]:
with open("data/star_wars/universe.yml") as f:
    config = f.read()
print(config)

parsed = dummy_driver.parse(native={"dev_conf": config})

---
universe:
  individuals:
  - affiliation: REBEL_ALLIANCE
    age: 57
    name: Obi-Wan Kenobi
  - affiliation: REBEL_ALLIANCE
    age: 19
    name: Luke Skywalker
  - affiliation: EMPIRE
    age: 42
    name: Darth Vader
  - affiliation: REBEL_ALLIANCE
    age: 896
    name: Yoda



In [11]:
import json
print(json.dumps(parsed.raw_value(), indent=4))

{}
